__качаем объявления о продаже квартир с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
place = 'sevastopol'
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2023-03-29 15:28:36 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2023-03-29 15:28:36 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2023-03-29 15:28:39 | AdsListParser: downloader init
[INFO    ] 2023-03-29 15:28:39 | CianParser: downloader init
[INFO    ] 2023-03-29 15:28:39 | AdsListParser: start read and parse pages...
[INFO    ] 2023-03-29 15:28:49 | AdsListParser: read page 1
[WARNING ] 2023-03-29 15:28:49 | CianParser: parse pagination error: 'NoneType' object has no attribute 'find_all'
[INFO    ] 2023-03-29 15:28:49 | AdsListParser: last page detected
[INFO    ] 2023-03-29 15:28:50 | DownloaderSeleniumFirefox: close virtual browser
28
CPU times: user 1.02 s, sys: 80.8 ms, total: 1.1 s
Wall time: 14.1 s


In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
16,"1-комн. кв., 53,3 м², 7/10 этаж",Секция 1 • Сдача корпуса 3 кв. 2022,сдача ГК: 4 кв. 2023 года,5 863 000 ₽,110 000 ₽/м²,"[Севастополь, р-н Ленинский, улица Генерала Крейзера, 8]","[10 часов назад, сегодня, 04:37]",https://sevastopol.cian.ru/sale/flat/281147191/,"СКИДКА ДО 25% НА ЛЮБУЮ КВАРТИРУ В ЖК*\n*подробности в отделе продаж застройщика\n\nСтатусный современный жилой комплекс бизнес-класса Центр находится в самом сердце города Федерального значения Севастополь. ЖК Центр жизнь в эпицентре событий. Расположение в историческом центре Ленинского района дарит возможность быстрого доступа до главных достопримечательностей города, важных социальных объектов, туристических и развлекательный мест, центральных набережных и пляжей.\nНа 1 этаже комплекса расположатся торгово-офисные помещения и ресторан. Три нижних этажа ЖК занимает подземный паркинг с видеонаблюдением, где Ваш автомобиль будет в целостности и сохранности.\nТри 10-этажных корпуса располагаются на едином стилобате это закрытый двор без машин под видеонаблюдением с детскими и спортивными площадками с безопасным покрытием, где Вы и Ваши дети могут чувствовать себя спокойно гуляя, играя и занимаясь спортом.\nХоллы комплекса выполнены по индивидуальному дизайн проекту, в каждом подъезде установлены бесшумные скоростные лифты, спускающиеся в паркинг.\nБольшинство квартир ЖК Центра видовые. В квартирах сделаны увеличенные оконные проемы либо установлены панорамные окна, что дарит много света и максимальный вид на панораму города, праздничные салюты, исторические достопримечательности и, конечно, морские пейзажи. \nКаждая квартира оборудована автономным двухконтурным газовым котлом, что экономит расходы на КУ и дает возможность контроля температуры в квартире под Ваши предпочтения.\nВ пешей доступности от комплекса лучшие гимназии города, несколько детских садов, поликлиник и больниц, спортивный оздоровительный комплекс 200 лет Севастополя, футбольное и баскетбольное поле, поле для Регби, , открытые теннисные корты, крытый бассейн, Ледовая арена для занятий фигурным катанием и хоккеем. В 5 минутах от комплекса находится рынок Чайка.",1,2023-03-29 15:28:49.835295
21,Продам 1кк с Новым ремонтом!!,"1-комн. кв., 42 м², 9/11 этаж",,6 290 000 ₽,149 762 ₽/м²,"[Севастополь, р-н Нахимовский, улица Горпищенко, 127к8]","[вчера, вчера, 21:55]",https://sevastopol.cian.ru/sale/flat/284802138/,"ШИКАРНАЯ КВАРТИРА С ПАНОРАМНЫМ ВИДОМ НА Горы\nСтильная, просторная 1-комнатная квартира на 9 этаже 10 этажного жилого дома Атмосфера Из окон открывается удивительный панорамный вид на Горы и лес. Бонус:всегда свежий воздух!\n\nОптимальная планировка: большая квадратная комната, светлая и просторная Кухня оборудованная необходимой техникой, просторная ванная комната. \n Ремонт новый состояние идеальное. Встроенная кухонная стенка, новая сантехника, металлопластиковые стеклопакеты, медная проводка, красивые межкомнатные двери, входная -поменяна. \n Уютный зеленый двор с удобными деревянными скамейками. Детская площадка с каруселями и горками, детский скалодром. Большой плюс ОГРОМНАЯ парковка. \n НОВЫЙ КРАСИВЫЙ ДОМ: кирпичный, теплый. Домофон,грузовой лифт, красивый подъезд.\nВ шаговой доступности расположены 2 школы, детсад, на первом этаже соседнего подъезда маленький магазин. Супермаркеты в шаговой доступности от дома, до остановки 5 мин.\nКвартира ждет нового хозяина. Всё сделано со вкусом и трепетом! Звоните! Увидев эту квартиру один раз, вы уже не захотите уходить!",1,2023-03-29 15:28:49.835295
6,"2-комн. кв., 61,63 м², 2/10 этаж",Сдача корпуса 4 кв. 2023,сдача ГК: 4 кв. 2023 года,6 163 000 ₽,100 000 ₽/м²,"[Севастополь, р-н Гагаринский, Казачья Бухта мкр, Скифия ЖК]","[вчера, вчера, 10:09]",https://sevastopol.cian.ru/sale/flat/281146471/,"Прекрасная квартира с городской пропиской в новом доме у моря. \n5 минут до пляжа! Остановка в шаговой доступности , 15 минут до центра города!\n\nДвухконту

In [7]:
assert len(df)>0

In [8]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

28
28


In [9]:
df['place']=place

In [10]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [11]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [12]:
# df

In [13]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [14]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [15]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])